In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive')
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/final/'
sys.path.append(project_dir+'services')

Mounted at /content/drive


In [2]:
%%writefile "/content/drive/MyDrive/Colab Notebooks/project/final/services/emotion_service.py"
"""
Emotion service for the music recommender app.
Handles emotion detection and processing.
"""
import pandas as pd
import numpy as np
import time
from IPython.display import display, HTML, clear_output

class EmotionService:
    """Handles emotion detection and processing."""

    def __init__(self, emotion_csv_path):
        """
        Initialize the emotion service.

        Args:
            emotion_csv_path (str): Path to the emotion results CSV file
        """
        self.emotion_csv_path = emotion_csv_path
        self.load_emotion_data()

    def load_emotion_data(self):
        """Load emotion data from CSV."""
        try:
            self.result_df = pd.read_csv(self.emotion_csv_path)
            # If the file exists but doesn't have the required column, add it
            if 'stress_total' not in self.result_df.columns:
                self.result_df['stress_total'] = np.random.uniform(1.0, 4.0, size=len(self.result_df))
                if self.result_df.empty:
                    # Create sample data if DataFrame is empty
                    self.result_df = pd.DataFrame({
                        'user_id': [1, 1, 1, 1],
                        'song_id': [1, 2, 3, 4],
                        'emotion_before': ['happy', 'sad', 'relaxed', 'anger'],
                        'emotion_after': ['relaxed', 'happy', 'happy', 'relaxed'],
                        'stress_total': [1.5, 3.0, 2.2, 3.8]
                    })
                self.result_df.to_csv(self.emotion_csv_path, index=False)
        except (FileNotFoundError, pd.errors.EmptyDataError):
            # Create sample DataFrame if file doesn't exist or is empty
            self.result_df = pd.DataFrame({
                'user_id': [1, 1, 1, 1],
                'song_id': [1, 2, 3, 4],
                'emotion_before': ['happy', 'sad', 'relaxed', 'anger'],
                'emotion_after': ['relaxed', 'happy', 'happy', 'relaxed'],
                'stress_total': [1.5, 3.0, 2.2, 3.8]
            })
            self.result_df.to_csv(self.emotion_csv_path, index=False)

    def save_emotion_data(self):
        """Save emotion data to CSV."""
        self.result_df.to_csv(self.emotion_csv_path, index=False)

    def detect(self, row_idx=None, show_ui=True):
        """
        Simulate emotion detection.
        Uses stress values to determine emotion.

        Args:
            row_idx (int, optional): Specific row to use for emotion. If None, randomly selected.
            show_ui (bool): Whether to display emotion detection UI

        Returns:
            str: Detected emotion
        """
        if row_idx is None:
            if self.result_df.empty:
                self.load_emotion_data()  # Reload or create sample data if empty
            row_idx = np.random.choice(self.result_df.index.tolist())

        if show_ui:
            self._show_loading_ui()

        # Detect emotion based on stress value
        try:
            stress_value = self.result_df.loc[row_idx, "stress_total"]

            if stress_value >= 3.5:
                emotion = "anger"
            elif stress_value >= 2.5:
                emotion = "sad"
            elif stress_value >= 2:
                emotion = "happy"
            else:
                emotion = "relaxed"
        except (KeyError, IndexError):
            # Fallback if row or column doesn't exist
            emotions = ['happy', 'sad', 'relaxed', 'anger']
            emotion = np.random.choice(emotions)

        if show_ui:
            self._show_emotion_ui(emotion)

        return emotion

    def _show_loading_ui(self):
        """Display a loading animation while detecting emotion."""
        clear_output(wait=True)
        display(HTML("""
        <div style="text-align:center;">
            <div style="width: 50px; height: 50px; background: #ff9900;
                        border-radius: 50%; animation: pulse 1s infinite; margin: 10px auto;"></div>
            <p style="font-size:16px;">Scanning for emotional signals...</p>
        </div>
        <style>
        @keyframes pulse {
            0% { transform: scale(1); opacity: 1; }
            50% { transform: scale(1.2); opacity: 0.7; }
            100% { transform: scale(1); opacity: 1; }
        }
        </style>
        """))
        time.sleep(2)

    def _show_emotion_ui(self, emotion):
        """
        Display a UI showing the detected emotion.

        Args:
            emotion (str): The detected emotion
        """
        clear_output(wait=True)

        emoji_map = {
            "happy": "😊",
            "sad": "😢",
            "anger": "😡",
            "relaxed": "😌"
        }
        emoji = emoji_map.get(emotion, "🎭")

        display(HTML(f"""
        <style>
        @keyframes fadeIn {{
            from {{ opacity: 0; transform: translateY(10px); }}
            to {{ opacity: 1; transform: translateY(0); }}
        }}
        </style>
        <div style="display: flex; justify-content: center; margin-top: 20px;">
            <div style="
                background-color: #f0f8ff;
                color: #006064;
                border-left: 5px solid #26c6da;
                border-radius: 8px;
                padding: 10px 20px;
                font-size: 18px;
                font-weight: bold;
                animation: fadeIn 1s ease-in-out;
                text-align: center;
            ">
                {emoji} <span style="font-weight:bold;">Emotion Detected:</span> {emotion}
            </div>
        </div>
        """))

    def record_emotion(self, user_id, song_id, emotion_before, emotion_after):
        """
        Record emotion data for a song play.

        Args:
            user_id (int): The user ID
            song_id (int): The song ID
            emotion_before (str): Emotion before playing the song
            emotion_after (str): Emotion after playing the song

        Returns:
            bool: True if successful
        """
        # Generate a random stress value based on emotion
        stress_map = {
            'relaxed': np.random.uniform(1.0, 2.0),
            'happy': np.random.uniform(2.0, 2.5),
            'sad': np.random.uniform(2.5, 3.5),
            'anger': np.random.uniform(3.5, 4.0)
        }
        stress_value = stress_map.get(emotion_before, np.random.uniform(1.0, 4.0))

        new_record = pd.DataFrame([{
            'user_id': user_id,
            'song_id': song_id,
            'emotion_before': emotion_before,
            'emotion_after': emotion_after,
            'stress_total': stress_value
        }])

        self.result_df = pd.concat([self.result_df, new_record], ignore_index=True)
        self.save_emotion_data()

        return True

    def get_mood_improving_songs(self, emotion='sad'):
        """
        Get songs that have improved mood from the specified emotion.

        Args:
            emotion (str): The emotion to improve from

        Returns:
            list: List of song IDs that have improved mood
        """
        if self.result_df.empty:
            return []

        # Filter for records where emotion_before matches and emotion_after is better
        better_emotions = ['happy', 'relaxed']
        improved_records = self.result_df[
            (self.result_df['emotion_before'] == emotion) &
            (self.result_df['emotion_after'].isin(better_emotions))
        ]

        if improved_records.empty:
            return []

        return improved_records['song_id'].unique().tolist()

    def get_user_emotion_history(self, user_id):
        """
        Get a user's emotion history.

        Args:
            user_id (int): The user ID

        Returns:
            pandas.DataFrame: The user's emotion history
        """
        if self.result_df.empty:
            return pd.DataFrame()

        return self.result_df[self.result_df['user_id'] == user_id]

Overwriting /content/drive/MyDrive/Colab Notebooks/project/final/services/emotion_service.py


In [ ]:
import pandas as pd
import random

class EmotionService:
    """Handles emotion detection and processing."""

    def __init__(self, emotion_csv_path):
        """
        Initialize the emotion service.

        Args:
            emotion_csv_path (str): Path to the emotion results CSV file
        """
        self.emotion_csv_path = emotion_csv_path
        self.load_emotion_data()

    def load_emotion_data(self):
        """Load emotion data from CSV."""
        try:
            self.emotion_df = pd.read_csv(self.emotion_csv_path)
        except (FileNotFoundError, pd.errors.EmptyDataError):
            # Create empty DataFrame if file doesn't exist or is empty
            self.emotion_df = pd.DataFrame(columns=[
                'user_id', 'song_id', 'emotion_before', 'emotion_after'
            ])

    def save_emotion_data(self):
        """Save emotion data to CSV."""
        self.emotion_df.to_csv(self.emotion_csv_path, index=False)

    def detect(self):
        """
        Simulate emotion detection.
        In a real app, this might use camera input or user feedback.

        Returns:
            str: Detected emotion
        """
        # Simplified for demonstration - in a real app,
        # this might analyze facial expressions or ask the user
        emotions = ['happy', 'sad', 'neutral', 'excited', 'relaxed']
        return random.choice(emotions)

    def record_emotion(self, user_id, song_id, emotion_before, emotion_after):
        """
        Record emotion data for a song play.

        Args:
            user_id (int): The user ID
            song_id (int): The song ID
            emotion_before (str): Emotion before playing the song
            emotion_after (str): Emotion after playing the song

        Returns:
            bool: True if successful
        """
        new_record = pd.DataFrame([{
            'user_id': user_id,
            'song_id': song_id,
            'emotion_before': emotion_before,
            'emotion_after': emotion_after
        }])

        self.emotion_df = pd.concat([self.emotion_df, new_record], ignore_index=True)
        self.save_emotion_data()

        return True

    def get_mood_improving_songs(self, emotion='sad'):
        """
        Get songs that have improved mood from the specified emotion.

        Args:
            emotion (str): The emotion to improve from

        Returns:
            list: List of song IDs that have improved mood
        """
        if self.emotion_df.empty:
            return []

        # Filter for records where emotion_before matches and emotion_after is better
        better_emotions = ['happy', 'excited', 'relaxed']
        improved_records = self.emotion_df[
            (self.emotion_df['emotion_before'] == emotion) &
            (self.emotion_df['emotion_after'].isin(better_emotions))
        ]

        if improved_records.empty:
            return []

        return improved_records['song_id'].unique().tolist()

    def get_user_emotion_history(self, user_id):
        """
        Get a user's emotion history.

        Args:
            user_id (int): The user ID

        Returns:
            pandas.DataFrame: The user's emotion history
        """
        if self.emotion_df.empty:
            return pd.DataFrame()

        return self.emotion_df[self.emotion_df['user_id'] == user_id]

Overwriting /content/drive/MyDrive/Colab Notebooks/project/final/services/emotion_service.py
